In [1]:
"""
FOR THIS FILE WE ARE ATTEMPTING TO TRAIN OUR MODEL AFTER GETTING OUR FEATURES
Implementing method outlined in this paper:
https://www-cs.stanford.edu/~acoates/papers/coatesng_nntot2012.pdf
"""

import numpy as np
from cv2 import cv2 #Use 'pip install opencv-python' to get module if you don't have it
import glob
import os
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

class GetImage(object):
    #Load raw image files
    
    #initialize the path (TODO - make basepath an input parameter, don't hardcode too much)

    #Data should be stored in Users/YOUR_USER_NAME/321_galaxies/images_training_rev1
    def __init__(self, num_pics=15000, username='mike'):
        self.num_pics = num_pics
        self.basepath = '/Users/'+username+'/321_galaxies/'
        self.imPath_test = 'images_test_rev1/'
        self.imPath_training =  'images_training_rev1/'
        self.solPath = 'training_solutions_rev1.csv'
        self.solutions=np.loadtxt(self.basepath + self.solPath, delimiter=',', skiprows=1)[:num_pics, 1:4]
        self.patchvector = []
       
        #create variable that stores the image
        self.images = []
        i=0
        for filename in glob.glob(self.basepath + self.imPath_training+'*.jpg'): #assuming jpg
            if(i >= num_pics):
                break
            im=cv2.imread(filename)
            self.images.append(im)
            i +=1 
        self.cropped_images = []
        self.scaled_images = []
        self.patches = []
       
    #Get category labels for each image from the solutions file
    def getLabels(self):
        self.label=[]
        for i in self.solutions:
            self.m = np.where(i == max(i))[0][0]
            self.label.append(self.m)
        self.label=np.array(self.label)
        return self
    
    #Crop image from 424x424 to 160x160. To do we crop axes from {x=0, y=0, x=424, y=424 } => {x=132, y=132, x=292, 400=292}
    def crop(self, pixels_to_keep = 160):
        for galaxyPic in self.images:
            #Cast as int in order to use variables in the slicing indices
            center = int(424/2)
            dim = int(pixels_to_keep/2)
            cropmin = center - dim
            cropmax = center + dim
            self.cropped_images.append(galaxyPic[cropmin:cropmax, cropmin:cropmax])
        return self

    #function that takes an image and scales the entire image to its new size (unlike crop, image stays intact). 
    def scale(self, new_size = 16):
        for galaxyPic in self.cropped_images:
            dimensions = (int(new_size), int(new_size))
            self.scaled_images.append(cv2.resize(galaxyPic, dimensions))
        return self
    
    #function that crops out the four 8x8 patches that compose each image.
    def patch(self, p_size=8):
        for galaxyPic in self.scaled_images:
            #Gets the center for each of the four patches
            for i in (4, 12):
                for j in (4, 12):
                    patch_x = i
                    patch_y = j
                    dim = int(p_size/2)
                    patchminx = patch_x - dim
                    patchminy = patch_y - dim
                    patchmaxx = patch_x + dim
                    patchmaxy = patch_y + dim
                    self.patches.append(galaxyPic[patchminx:patchmaxx, patchminy:patchmaxy])
        return self
    #function that sums the values for the different filters and
    #flatten the 4x4 matrixes for each patch to a 1x16 vector
    def makevector(self):
        self.newpatches = np.sum(self.patches, axis = 3)
        
        for i in self.newpatches:
            self.patchvector.append(np.matrix.flatten(i).tolist())
        self.patchvector=np.array(self.patchvector)
        return self
    
        
    ## Function that normalizes the pixel values. Takes vstack of patches as input
    def normalize(self):
        temp1 = self.patchvector - self.patchvector.mean(1, keepdims=True)
        temp2 = np.sqrt(self.patchvector.var(1, keepdims=True) + 10)
        self.patchvector = temp1/temp2
        return self
    ## Function to perform ZCA whitening to decorrelate our pixels
    def whiten(self):
        cov = np.cov(self.patchvector, rowvar=0)
        self.mean = self.patchvector.mean(0, keepdims=True)
        d, v = np.linalg.eig(cov)
        self.p = np.dot(v, np.dot(np.diag(np.sqrt(1 / (d + 0.1))), v.T))
        self.patchvector = np.dot(self.patchvector - self.mean, self.p)
        return self



class extractFeatures():

    def __init__(self, vectorPatches, k = 100):
        #Making a dictionnary of centroids from our patch vectors
        self.k = k
        self.patches = vectorPatches
        self.dictionary = self.Kmeans()
        #We then use this dictionnary to create features for each patch vector
        self.features = []
        for patch in self.patches:
            self.features.append(np.matmul(self.dictionary, patch))
        self.features = np.array(self.features)

    def getActualFeatures(self):
        self.truefeatures=[]
        for i in range(int(len(self.features) / 4)) :
            self.truefeatures.append((self.features[4*i], self.features[4*i+1], self.features[4*i+2], self.features[4*i+3]))
        self.truefeatures = np.array(self.truefeatures)
        return self.truefeatures

    def Kmeans(self, iters = 25, batch_size = 1000):
        L2 = np.sum(self.patches**2, 1, keepdims=True)
        #initialize centroids
        self.centroids = np.random.randn(self.k, self.patches.shape[1]) * 0.1
        for iteration in range(1, iters+1):
            c2 = np.sum(self.centroids**2, 1, keepdims=True)
            summation = np.zeros((self.k, self.patches.shape[1]))
            counts = np.zeros((self.k, 1))
            loss = 0
            for i in range(0, self.patches.shape[0], batch_size):
                last_index = min(i + batch_size, self.patches.shape[0])
                m = last_index - i

                # shape (k, batch_size) - shape (k, 1)
                tmp = np.dot(self.centroids, self.patches[i:last_index, :].T) - c2
                # shape (batch_size, )
                indices = np.argmax(tmp, 0)
                # shape (1, batch_size)
                val = np.max(tmp, 0, keepdims=True)

                loss += np.sum((0.5 * L2[i:last_index]) - val.T)

                # Don't use a sparse matrix here
                S = np.zeros((batch_size, self.k))
                S[range(batch_size), indices] = 1

                # shape (k, n_pixels)
                this_sum = np.dot(S.T, self.patches[i:last_index, :])
                summation += this_sum

                this_counts = np.sum(S, 0, keepdims=True).T
                counts += this_counts 
            
            self.centroids = summation / counts
        
            bad_indices = np.where(counts == 0)[0]
            self.centroids[bad_indices, :] = 0
        return self.centroids


class classifyGalaxies():

    def __init__(self, galaxyPics, featureExtractor, percentForTrain = 0.75):

        self.galaxyPics = galaxyPics
        self.num_pics = galaxyPics.num_pics
        self.truefeatures = featureExtractor.getActualFeatures()
        ##Reshaping our features for our multiclass algorithm in the next cell
        self.nsamples, self.nx, self.ny = self.truefeatures.shape
        self.truefeatures = self.truefeatures.reshape((self.nsamples,self.nx*self.ny))
        self.partition = int(percentForTrain*self.num_pics)
        ###Separating the training data and test data for our multiclass model
        self.train_x = self.truefeatures[:self.partition]
        self.train_y = galaxyPics.label[:self.partition]
        self.test_x = self.truefeatures[self.partition:self.num_pics]
        self.test_y = galaxyPics.label[self.partition:self.num_pics]

    ### Function that tests the accuracy using the Root Mean Squared Method
###Here we use the probabilities that an image belongs in a cluster to get a better
###value of accuracy
    def testRMS(self, preds, labels):
        count = 0
        N = len(preds) * 3
        for i in range(len(preds)):
            for j in range(len(preds[0])):
                count += (preds[i][j] - labels[i][j])**2
        return 100*(1-np.sqrt(count/N))
    

    def classify(self, useSVC = True, useGradientBoosting = True, useRadiusNeighbors = True, useRandomForest = True, useLogisticRegression = True):
        if(useSVC):
            model=OneVsRestClassifier(SVC(random_state=0, max_iter=50000, probability=True)).fit(self.train_x, self.train_y)
            predictions = model.predict(self.test_x)
            predictproba = model.predict_proba(self.test_x)
            print("The root mean square accuracy using SVC is: " )
            print(self.testRMS(predictproba, self.galaxyPics.solutions[self.partition:self.num_pics]))

        if(useGradientBoosting):
            model = GradientBoostingClassifier(n_estimators=300, learning_rate=0.001, max_depth=3, random_state=0).fit(self.train_x, self.train_y)
            pred = model.predict_proba(self.test_x)
            print("\n The root mean square accuracy using Gradient Boosting is: ") 
            print(self.testRMS(pred, self.galaxyPics.solutions[self.partition:self.num_pics]))

        if(useRadiusNeighbors):
            neigh = RadiusNeighborsClassifier(radius=2000.0)
            neigh.fit(self.train_x, self.train_y)
            pred = neigh.predict_proba(self.test_x)
            print("\n The root mean square accuracy using Radius Neighbors is: ")
            print(self.testRMS(pred, self.galaxyPics.solutions[self.partition:self.num_pics]))

        if(useRandomForest):
            clf2 = RandomForestClassifier(max_depth=2, random_state=0)
            clf2.fit(self.train_x, self.train_y)
            pred4 = clf2.predict_proba(self.test_x)
            print("\n The root mean square accuracy using Random Forest is: ") 
            print(self.testRMS(pred4, self.galaxyPics.solutions[self.partition:self.num_pics]))

        if(useLogisticRegression):
            model4 = LogisticRegression(random_state=0, max_iter=50000).fit(self.train_x, self.train_y)
            predict2 = model4.predict_proba(self.test_x)
            print("\n The root mean square accuracy using Logistic Regression is: " )
            print(self.testRMS(predict2, self.galaxyPics.solutions[self.partition:self.num_pics]))

In [2]:

#Testing the class by creating a 'galaxyPic' object, cropping it, scaling it, extracting patches, making the 
#patch vectors, normalizing and whitening the pixel values. 
galaxyPics = GetImage()
galaxyPics.getLabels()
galaxyPics.crop()
galaxyPics.scale()
galaxyPics.patch()
galaxyPics.makevector()
galaxyPics.normalize()
galaxyPics.whiten()

featureClass = extractFeatures(galaxyPics.patchvector)

features = featureClass.getActualFeatures()

classifier = classifyGalaxies(galaxyPics, featureClass)

classifier.classify()

<ipython-input-1-bc309c43aece>:169: RuntimeWarning: invalid value encountered in true_divide
  self.centroids = summation / counts


The root mean square accuracy using SVC is: 
76.28223624710746

 The root mean square accuracy using Gradient Boosting is: 
76.29742525808594

 The root mean square accuracy using Radius Neighbors is: 
76.27531827825024

 The root mean square accuracy using Random Forest is: 
76.28130197271764

 The root mean square accuracy using Logistic Regression is: 
76.15047551673597


In [ ]:
#Running again with 200 centroids
galaxyPics = GetImage()
galaxyPics.getLabels()
galaxyPics.crop()
galaxyPics.scale()
galaxyPics.patch()
galaxyPics.makevector()
galaxyPics.normalize()
galaxyPics.whiten()

featureClass = extractFeatures(galaxyPics.patchvector, k=200)

features = featureClass.getActualFeatures()

classifier = classifyGalaxies(galaxyPics, featureClass)

classifier.classify()
